In [98]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

"""
Simple Moving Average
https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages
"""
def sma(data,period,column_name):
    data['SMA_'+ str(period)] = data[column_name].rolling(window=period).mean()
    return data


"""
Exponential Moving Average
https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages
"""
def ema(data,period,column_name):
    data['EMA_'+ str(period)] = data[column_name].ewm(ignore_na=False,min_periods=period, span=period,adjust=True).mean()
    return data

"""
Moving Average Convergence/Divergence Oscillator (MACD)
Source: http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_average_convergence_divergence_macd
"""
def macd(data,period_short,period_long,period_signal,column_name):
    # columns to drop
    cols2drop=[]
    
    # EMA short
    data['EMA_'+ str(period_short)] = data[column_name].ewm(ignore_na=False,min_periods=period_short, span=period_short,adjust=True).mean()
    ema_short=data['EMA_'+ str(period_short)]
    cols2drop.append('EMA_'+ str(period_short))
    
    # EMA long
    data['EMA_'+ str(period_long)] = data[column_name].ewm(ignore_na=False,min_periods=period_long, span=period_long,adjust=True).mean()
    ema_long=data['EMA_'+ str(period_long)]
    cols2drop.append('EMA_'+ str(period_long))

    # MACD Formula
    data_macd=ema_short - ema_long
    data['MACD_'+ str(period_short)+'_'+ str(period_long)] = data_macd
    data['MACD_signal_'+ str(period_signal)] = data_macd.ewm(ignore_na=False,min_periods=period_signal, span=period_signal,adjust=True).mean()
    
    # Drop Cols
    data.drop(cols2drop,axis=1, inplace=True)
    
    return data
"""
    Accumulation Distribution 
Source: http://stockcharts.com/school/doku.php?st=accumulation+distribution&id=chart_school:technical_indicators:accumulation_distribution_line
Params: 
    data: pandas DataFrame
    trend_periods: the over which to calculate AD
    open_col: the name of the OPEN values column
	high_col: the name of the HIGH values column
	low_col: the name of the LOW values column
	close_col: the name of the CLOSE values column
	vol_col: the name of the VOL values column
    
Returns:
    copy of 'data' DataFrame with 'acc_dist' and 'acc_dist_ema[trend_periods]' columns added
"""
def acc_dist(data,period,open_col,high_col,low_col,close_col,vol_col):
    data['Acc_Dist']=float(-666) 
    acidx=data.columns.get_loc("Acc_Dist")
    for index,row in data.iterrows():
        if row[high_col] != row[low_col]:
            mfm=((row[close_col] - row[low_col]) - (row[high_col] - row[close_col])) / (row[high_col] - row[low_col])
            mfv=mfm * row[vol_col]
        else:
            mfv=0
        data.iat[index,acidx]=mfv
    data['Acc_Dist_Line_' + str(period)] = data['Acc_Dist'].ewm(ignore_na=False,min_periods=0, com=period, adjust=True).mean()

    return data   
        
# Creating data array, Close first
b=np.arange(4,102,2) 
c=np.arange(102,4,-2)
d=[b,c]
d_array=np.resize(d,(1,(len(b)+len(c))))
close_col=d_array[0]    #CLOSE
# Randoms
np.random.seed(2)
rnd1=np.random.rand(1,(len(b)+len(c)))
rnd2=np.around(rnd1,5)
rnd1=np.around(rnd1,2)
# Open,High,Low,Vol
open_col=d_array-1
open_col=open_col[0]    #OPEN
high_col=open_col+1+abs(rnd1)*2
high_col=high_col[0]    #HIGH
low_col=open_col-abs(rnd1)*1
low_col=low_col[0]      #LOW
vol_col=abs(rnd2)*20000
vol_col=vol_col[0]      #VOL


df = pd.DataFrame({'OPEN': open_col,'CLOSE': close_col,'HIGH': high_col,'LOW': low_col,'VOL': vol_col})
sma_25=sma(df,25,'CLOSE')
ema_25=ema(df,25,'CLOSE')
macd_5_8=macd(df,26,12,9,'CLOSE')
acc_dist=acc_dist(df,50,'OPEN','HIGH','LOW','CLOSE','VOL')

df.head(80)

,OPEN,CLOSE,HIGH,LOW,VOL,SMA_25,EMA_25,MACD_26_12,MACD_signal_9,Acc_Dist,Acc_Dist_Line_50
0,3,4,4.88,2.56,8719.8,NaN,NaN,NaN,NaN,2104.779310,2104.779310
1,5,6,6.06,4.97,518.6,NaN,NaN,NaN,NaN,461.506422,1275.007852
2,7,8,9.10,6.45,10993.2,NaN,NaN,NaN,NaN,1866.769811,1476.180621
3,9,10,10.88,8.56,8706.4,NaN,NaN,NaN,NaN,2101.544828,1637.195812
4,11,12,12.84,10.58,8407.4,NaN,NaN,NaN,NaN,2157.651327,1745.449740
5,13,14,14.66,12.67,6606.6,NaN,NaN,NaN,NaN,2224.332663,1829.266370
6,15,16,16.40,14.80,4093.0,NaN,NaN,NaN,NaN,2046.500000,1862.173422
7,17,18,19.24,16.38,12385.4,NaN,NaN,NaN,NaN,1645.612587,1833.190406
8,19,20,20.60,18.70,5993.0,NaN,NaN,NaN,NaN,2207.947368,1878.203655
9,21,22,22.54,20.73,5336.6,NaN,NaN,NaN,NaN,2152.330387,1908.122848
